In [ ]:
import pandas as pd
e2 = pd.read_csv("../../Out_E2_V2.csv",index_col=[0])#.drop_duplicates()
e2

In [ ]:
import matplotlib.pyplot as plt

d = e2.copy()

metrics = ['max_dist', 'MSE', 'MAE']
metric_names = ['max. distance', 'MSE', 'MAE']
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

df_list = []

for i, metric in enumerate(metrics):
    avg_metric_by_degree = d.groupby('method')[metric].mean()
    df_list.append(avg_metric_by_degree)
    axs[i].plot(avg_metric_by_degree.index, avg_metric_by_degree.values, marker='o', linestyle='-')
    axs[i].set_xlabel('Method')
    axs[i].set_ylabel('Mean ' + metric_names[i])
    axs[i].set_title('Mean ' + metric_names[i])  # + ' for degrees 0 to ' + str(max(dataframe['degree'].unique())))
    axs[i].set_xticks(list(avg_metric_by_degree.index))
    axs[i].grid(True)

plt.tight_layout(pad=3)

plt.show()

df = pd.concat(df_list,axis=1)
print(df)

# for each method: accuracy measures (max_dist, MSE, MAE)
Question: How does our method compare to other methods for time series approximation and/or compression? Can it be improved by including an L1 fitting?

In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.8f}'.format)
mean_values = e2.groupby('method').agg({'max_dist': 'mean', 'MSE': 'mean', 'MAE': 'mean'}).reset_index()
mean_values

In [ ]:
"""import seaborn as sns
import matplotlib.pyplot as plt

# Get unique values of "method"
methods = e2['method'].unique()

# Create box plots
plt.figure(figsize=(15, 5))

# Loop through each metric
metrics = ['max_dist', 'MSE', 'MAE']
for i, metric in enumerate(metrics, start=1):
    plt.subplot(1, 3, i)
    sns.boxplot(data=e2, x='method', y=metric, order=methods)
    plt.xlabel('Method')
    plt.ylabel(metric)
    plt.title(f'Box plot for {metric}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()"""


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'axes.titlesize':12, 'axes.labelsize':12})

metrics = ['max_dist', 'MAE', 'MSE']
metric_names = ['Maximum Distance','MAE', 'MSE']

for i,metric in enumerate(metrics):
    plt.figure(figsize=(4,4))
    plt.bar(mean_values['method'], mean_values[metric], color='bgr'[metrics.index(metric)])
    #plt.xlabel('Methods')
    plt.ylabel(f'Mean {metric_names[i]}')
    plt.title(f'Mean {metric_names[i]} per method')
    plt.xticks(rotation=45)
    plt.xticks(range(6),["DFT",r'$L_\infty$',r'$L_\infty^*$',"LSQ","PAA","PLA"])
    plt.tight_layout()
    plt.savefig(f"../plots/e2/plot_mean_{metric}.pdf")
    plt.show()


In [ ]:
"""import matplotlib.pyplot as plt

# Plot mean values
plt.figure(figsize=(10, 6))

plt.bar(mean_values['method'], mean_values['max_dist'], color='b', label='Mean max. distance')
plt.bar(mean_values['method'], mean_values['MAE'],color='g', label='Mean MAE')
plt.bar(mean_values['method'], mean_values['MSE'],color='r', label='Mean MSE')
plt.xlabel('Methods')
plt.ylabel('Mean Value')
plt.title('Mean max. distance, MSE, and MAE per method')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()"""


In [ ]:
"""import matplotlib.pyplot as plt
import numpy as np

methods = mean_values['method']
metrics = ['max_dist','MSE', 'MAE', ]

bar_width = 0.2
num_methods = len(methods)
r = np.arange(num_methods)

fig, axs = plt.subplots(len(metrics), figsize=(10, 15))

for i, metric in enumerate(metrics):
    mean_val = mean_values[metric]
    metric_without_outliers = mean_values[f'{metric}']

    r1 = r + bar_width/2

    axs[i].bar(r1, mean_val, color='b', width=bar_width)#, label='?')

    axs[i].set_xlabel('Method', fontweight='bold')
    axs[i].set_ylabel(f'Mean {metric}', fontweight='bold')
    axs[i].set_xticks([r + bar_width/2 for r in range(num_methods)])
    axs[i].set_xticklabels(methods, rotation=45, ha='right')
    axs[i].set_title(f'Comparison of mean {metric}')
    #axs[i].legend()

plt.tight_layout()
plt.show()"""

# for each method: outliers vs. no outliers
Question: Does the removal of outliers increase the accuracy of the approximated time series? 
(If so: outliers need to be saved separately!)

In [ ]:
df_no_outliers = e2[e2['num_outliers_removed'] > 0]
print(len(df_no_outliers.index))
df_with_outliers = e2[e2['num_outliers_removed'] == 0]
print(len(df_with_outliers.index))

mean_values_no_outliers = df_no_outliers.groupby('method').agg({
    'max_dist': 'mean',
    'MSE': 'mean', 
    'MAE': 'mean'}).reset_index()
mean_values_no_outliers.rename(columns={
    'max_dist': 'max_dist w/o outliers', 
    'MSE': 'MSE w/o outliers', 
    'MAE': 'MAE w/o outliers'}, inplace=True)

mean_values_with_outliers = df_with_outliers.groupby('method').agg({
    'max_dist': 'mean', 
    'MSE': 'mean', 
    'MAE': 'mean'}).reset_index()

mean_values_with_outliers.rename(columns={
    'max_dist': 'max_dist', 
    'MSE': 'MSE', 
    'MAE': 'MAE'}, inplace=True)

mean_values_combined = pd.merge(mean_values_with_outliers, mean_values_no_outliers, on='method', how='outer')
mean_values_combined

In [ ]:
print(mean_values_combined.to_latex())

In [ ]:
"""import matplotlib.pyplot as plt
import numpy as np


# Define light and dark shades of blue, green, and red
light_colors = ['#ADD8E6', '#90EE90', '#FFB6C1']  # Light blue, light green, light pink
dark_colors = ['#0000FF', '#008000', '#FF0000']  # Dark blue, dark green, dark red


methods = mean_values_combined['method']
metrics = ['max_dist', 'MAE', 'MSE']
metric_names = ['max. distance','MAE', 'MSE']

bar_width = 0.2
num_methods = len(methods)
r = np.arange(num_methods)

#fig, axs = plt.subplots(len(metrics), figsize=(10, 15))
fig, axs = plt.subplots(len(metrics), figsize=(8,12))

for i, metric in enumerate(metrics):
    metric_with_outliers = mean_values_combined[metric]
    metric_without_outliers = mean_values_combined[f'{metric} w/o outliers']

    r1 = r + bar_width/2
    r2 = [x + bar_width for x in r1]

    # Bar plot with outliers (light shade)
    axs[i].bar(r1, metric_with_outliers, color=light_colors[i], width=bar_width, label='With Outliers')

    # Bar plot without outliers (dark shade)
    axs[i].bar(r2, metric_without_outliers, color=dark_colors[i], width=bar_width, label='Without Outliers')



    #axs[i].bar(r1, metric_with_outliers, color='b', width=bar_width, label='With Outliers')
    #axs[i].bar(r2, metric_without_outliers, color='r', width=bar_width, label='Without Outliers')

    axs[i].set_xlabel('Method', fontweight='bold')
    axs[i].set_ylabel(f'Mean {metric_names[i]}', fontweight='bold')
    axs[i].set_xticks([r + bar_width/2 for r in range(num_methods)])
    axs[i].set_xticklabels(["DFT",r'$L_\infty$',r'$L_\infty \text{ and } L_1$',"LSQ","PAA","PLA"], rotation=45, ha='right')
    axs[i].set_title(f'Mean {metric_names[i]} with and without Outlier Removal')
    axs[i].legend()
    plt.savefig(f"../plots/e2/compare_w_wo_outliers_{metric}.pdf")

plt.tight_layout()



plt.show()"""

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Define light and dark shades of blue, green, and red
light_colors = ['#ADD8E6', '#90EE90', '#FFB6C1']  # Light blue, light green, light pink
dark_colors = ['#0000FF', '#008000', '#FF0000']  # Dark blue, dark green, dark red


methods = mean_values_combined['method']
metrics = ['max_dist', 'MAE', 'MSE']
metric_names = ['Maximum Distance','MAE', 'MSE']

bar_width = 0.2
num_methods = len(methods)
r = np.arange(num_methods)

for i, metric in enumerate(metrics):

    metric_with_outliers = mean_values_combined[metric]
    metric_without_outliers = mean_values_combined[f'{metric} w/o outliers']

    r1 = r + bar_width/2
    r2 = [x + bar_width for x in r1]
    
    plt.figure(figsize=(6,4))  # Create a new figure for each metric

    # Bar plot with outliers (light shade)
    plt.bar(r1, metric_with_outliers, color=light_colors[i], width=bar_width, label='Original Time Series')

    # Bar plot without outliers (dark shade)
    plt.bar(r2, metric_without_outliers, color=dark_colors[i], width=bar_width, label='With Imputed Outliers')

    #plt.xlabel('Method', fontweight='bold')
    plt.ylabel(f'Mean {metric_names[i]}')#, fontweight='bold')
    plt.xticks([r + bar_width/2 for r in range(num_methods)], ["DFT",r'$L_\infty$',r'$L_\infty^*$',"LSQ","PAA","PLA"], rotation=45, ha='right')
    plt.title(f'Mean {metric_names[i]}: \n Original Time Series vs. Time Series with Imputed Outliers')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"../plots/e2/compare_w_wo_outliers_{metric}.pdf")
    plt.show()  # Show each figure individually


# for L8-L1: correlation between accuracy measures and time series properties
Question: Does our method approximate time series with certain properties more/less accurately?

In [ ]:
import seaborn as sns

In [ ]:
df_max_l1 = e2[e2['method'] == 'L8 and L1']

In [ ]:
"""correlation_matrix = df_max_l1[
    ['max_dist', 'MSE', 'MAE', 'num_data_pts', 'mean_diff', 'max_fst_derivative', 'max_snd_derivative']].corr()
print("Correlation Matrix:")
correlation_matrix"""

In [ ]:
#df_max_l1.iloc[:,2:]

In [ ]:
"""import seaborn as sns
sns.set_theme(style="ticks")
sns.pairplot(df_max_l1[['num_data_pts','method','max_dist','mean_diff','max_fst_derivative']],hue="method")
#sns.pairplot(df_max_l1[2:],hue="method")"""

In [ ]:
for measurement in ['max_fst_derivative','max_snd_derivative','avg_fst_derivative','avg_snd_derivative','num_outliers_removed','num_data_pts']:
    rel = sns.relplot(data=df_max_l1, x=measurement, y="max_dist")
    rel.savefig(f"../plots/e2/sns_relplot_md_vs_{measurement}.png")

In [ ]:
g = sns.PairGrid(df_max_l1[['num_outliers_removed','num_data_pts','max_dist','MAE','MSE','max_fst_derivative','max_snd_derivative','avg_fst_derivative','avg_snd_derivative']]).map_offdiag(sns.scatterplot).map_diag(sns.histplot)
g.savefig("../plots/e2/sns_pairgrid.png")
#g = sns.PairGrid(df_max_l1[['num_data_pts','max_dist','max_fst_derivative','max_snd_derivative','avg_fst_derivative','avg_snd_derivative']]).map(sns.scatterplot)

In [ ]:
"""import seaborn as sns
sns.set_theme(style="ticks")
# for method in ['L8', 'L8 and L1', 'LSQ', 'PAA', 'PLA', 'DFT']:
df = e2[e2['method']=='L8 and L1']
#sns.pairplot(df[['num_data_pts','method','max_dist','mean_diff','max_fst_derivative']],hue="method")
sns.pairplot(df[2:],hue="method")"""

In [ ]:
"""for measurement in ['max_fst_derivative','max_snd_derivative','avg_fst_derivative','avg_snd_derivative','num_outliers_removed','num_data_pts']:
    reg = sns.regplot(data=df_max_l1, x=measurement, y="max_dist")
    reg.savefig(f"../plots/e2/sns_regplot_md_vs_{measurement}.png")"""

# Pearson Corr. Coeff. 

In [ ]:
#np.finfo(np.float64).tiny

In [ ]:
from scipy.stats import pearsonr
corr_coeff, pvalue = pearsonr(df_max_l1['max_dist'], df_max_l1['avg_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
corr_coeff, pvalue = pearsonr(df_max_l1['max_dist'], df_max_l1['avg_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
corr_coeff, pvalue = pearsonr(df_max_l1['max_dist'], df_max_l1['max_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
corr_coeff, pvalue = pearsonr(df_max_l1['max_dist'], df_max_l1['max_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
sns.residplot(x=df_max_l1['max_dist'], y=df_max_l1['avg_fst_derivative'],order=3)#,lowess=True,order=3)

# Other Methods
## PLA

In [ ]:
df_paa = e2[e2['method'] == 'PAA']
print(len(df_paa.index))

df_pla = e2[e2['method'] == 'PLA']
print(len(df_pla.index))
      
df_l8 = e2[e2['method'] == 'L8']
print(len(df_l8.index))

df_dft = e2[e2['method'] == 'DFT']
print(len(df_dft.index))

df_lsq = e2[e2['method'] == 'LSQ']
print(len(df_lsq.index))

In [ ]:
print("PAA")
corr_coeff, pvalue = pearsonr(df_paa['max_dist'], df_paa['avg_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_paa['max_dist'], df_paa['avg_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_paa['max_dist'], df_paa['max_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_paa['max_dist'], df_paa['max_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
print("PLA")
corr_coeff, pvalue = pearsonr(df_pla['max_dist'], df_pla['avg_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_pla['max_dist'], df_pla['avg_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_pla['max_dist'], df_pla['max_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_pla['max_dist'], df_pla['max_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
print("L8")
corr_coeff, pvalue = pearsonr(df_l8['max_dist'], df_l8['avg_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_l8['max_dist'], df_l8['avg_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_l8['max_dist'], df_l8['max_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_l8['max_dist'], df_l8['max_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
print("LSQ")
corr_coeff, pvalue = pearsonr(df_lsq['max_dist'], df_lsq['avg_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_lsq['max_dist'], df_lsq['avg_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_lsq['max_dist'], df_lsq['max_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_lsq['max_dist'], df_lsq['max_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
print("DFT")
corr_coeff, pvalue = pearsonr(df_dft['max_dist'], df_dft['avg_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_dft['max_dist'], df_dft['avg_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_dft['max_dist'], df_dft['max_fst_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

corr_coeff, pvalue = pearsonr(df_dft['max_dist'], df_dft['max_snd_derivative'])
print("correlation coefficient:", corr_coeff)
print("p-value:", pvalue)

In [ ]:
"""sns.PairGrid(df_pla[['num_outliers_removed','num_data_pts','max_dist','MAE','MSE','max_fst_derivative','max_snd_derivative','avg_fst_derivative','avg_snd_derivative']]).map_offdiag(sns.scatterplot).map_diag(sns.histplot)"""